In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povms.library.pm_sim_implementation import ClassicalShadows, RandomizedPMs
from povms.post_processor.povm_post_processor import POVMPostprocessor
from povms.sampler.povm_sampler import POVMSampler
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

# Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
n_qubit = 2
n_cicuits = 13
np.random.seed(10)
seeds= np.random.randint(0,10000000,size=n_cicuits)
qc_random = [random_circuit(num_qubits=n_qubit, depth = 3, measure=False, seed=seed) for seed in seeds]
qc_random[0].draw()

┌────────────────┐┌─────┐┌─────┐
q_0: ┤ R(3.871,5.133) ├┤ Sdg ├┤ Sdg ├
     └─────┬───┬──────┘└──┬──┘└┬───┬┘
q_1: ──────┤ Z ├──────────■────┤ T ├─
           └───┘               └───┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exp_val = np.zeros((len(qc_random), len(set_observables)), dtype=complex)
for i, qc in enumerate(qc_random):
    exact_state = Statevector(np.array(Operator(qc).data[:,0]).squeeze())
    exp_val[i] = np.array([exact_state.expectation_value(obs) for obs in set_observables])
exp_val = np.real_if_close(exp_val)

# Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
cs_shots = 4096

In [7]:
pubs = []
for qcr in qc_random:
    pubs.append((qcr, None, np.random.randint(100000,1000000) if np.random.randint(2)==0 else None, cs_implementation))
print(pubs)

[(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x111055810>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16637da20>, None, 940036, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16637e200>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16637e320>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16637dea0>, None, 922616, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16637d870>, None, None, <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>), (<qiskit.circuit.quantu

In [8]:
# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(pubs, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:156: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


## Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [9]:
cs_result = cs_job.result()
sum(cs_result[1].get_counts().values())

print(len(cs_job.book_keeping), cs_job.book_keeping)
print(len(cs_job.pvm_keys),cs_job.pvm_keys)
raw_result = cs_job.base_job.result()
len(raw_result)
for i in range(len(cs_job.pvm_keys)):
    print(raw_result[cs_job.book_keeping[i]])
    print(cs_job.pvm_keys[i])

13 [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
13 [[(0, 0), (1, 2), (2, 2), (1, 0), (1, 1), (2, 1), (0, 1), (0, 2), (2, 0)], [(1, 0), (2, 1), (1, 2), (2, 2), (0, 2), (0, 0), (0, 1), (2, 0), (1, 1)], [(0, 2), (1, 2), (1, 1), (0, 0), (1, 0), (0, 1), (2, 1), (2, 2), (2, 0)], [(1, 0), (1, 1), (2, 0), (0, 0), (2, 2), (0, 2), (2, 1), (1, 2), (0, 1)], [(2, 1), (0, 2), (0, 1), (2, 2), (2, 0), (1, 1), (0, 0), (1, 0), (1, 2)], [(0, 0), (1, 0), (1, 2), (2, 2), (2, 0), (2, 1), (1, 1), (0, 2), (0, 1)], [(0, 2), (0, 0), (0, 1), (2, 2), (2, 1), (1, 1), (1, 0), (2, 0), (1, 2)], [(1, 1), (1, 0), (2, 0), (2, 2), (0, 1), (2, 1), (0, 0), (1, 2), (0, 2)], [(2, 2), (0, 2), (1, 2), (0, 1), (0, 0), (1, 1), (1, 0), (2, 0), (2, 1)], [(0, 1), (1, 0), (1, 1), (2, 0), (2, 2), (0, 2), (0, 0), (1, 2), (2, 1)], [(0, 1), (0, 0), (2, 1), (2, 2), (2, 0), (0, 2), (1, 1), (1, 2), (1, 0)], [(2, 2), (1, 0), (1, 1), (2, 0), (0, 2), (1, 2), (0, 1), (2, 1), (0, 0)], [(1, 2), (1, 0), (1, 1), (0, 1), (2, 1), (0, 0), (2, 2), (0, 2), 

In [10]:
cs_result = cs_job.result()
print(f"POVM: {cs_result[0].povm}")
print(f"Counts: {cs_result[1].get_counts()}")

POVM: <povms.library.pm_sim_implementation.ClassicalShadows object at 0x111056dd0>
Counts: {(2, 0): 52106, (3, 0): 52365, (4, 3): 25956, (4, 2): 26420, (5, 3): 25946, (5, 2): 26091, (2, 5): 26250, (3, 5): 25869, (2, 4): 26134, (3, 4): 25984, (4, 5): 26119, (5, 4): 26025, (4, 4): 26428, (5, 5): 26208, (0, 4): 52078, (0, 5): 52381, (0, 0): 104332, (0, 3): 51844, (0, 2): 52099, (5, 0): 52464, (4, 0): 52397, (3, 2): 25883, (2, 2): 26361, (2, 3): 26115, (3, 3): 26181}


We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [11]:
cs_postprocessor = POVMPostprocessor(cs_result[0])
cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]

In [12]:
for i_state in range(len(cs_result)):
    cs_postprocessor = POVMPostprocessor(cs_result[i_state])
    cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]
    cs_shots = sum(cs_postprocessor.counts.values())

    n = int(np.ceil(np.log10(cs_shots)))
    print('Exact        CS')
    print(f'             ({cs_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {abs(cs_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        CS
             (4096 shots)
 2.244e+00      2.387e+00       6.4%
-1.221e+00     -1.312e+00       7.5%
-8.662e-01     -1.008e+00      16.4%
 9.594e-01      1.067e+00      11.2%
-1.975e-01     -2.688e-01      36.1%
-7.249e-01     -7.894e-01       8.9%
 3.303e-01      1.678e-01      49.2%
-1.498e-01     -1.571e-01       4.9%
 1.441e+00      1.542e+00       7.0%
-5.073e-01     -7.488e-01      47.6%
Exact        CS
             (940036 shots)
 1.582e-01        1.584e-01     0.1%
 6.594e-02        6.268e-02     4.9%
 3.723e-01        3.698e-01     0.7%
 2.555e-01        2.566e-01     0.4%
-4.867e-01       -4.837e-01     0.6%
-7.440e-01       -7.462e-01     0.3%
 2.420e+00        2.423e+00     0.1%
 3.360e-01        3.342e-01     0.5%
-6.905e-02       -6.018e-02    12.8%
 8.534e-01        8.561e-01     0.3%
Exact        CS
             (4096 shots)
 2.427e+00      2.394e+00       1.4%
-9.346e-02     -9.477e-02       1.4%
 4.495e-01      5.059e-01      12.5%
-2.687e-02     -4.399

# PM-simulable POVM
We now look at POVM that are simulable (through randomization) with only single-qubit projective measurements (PMs).

In [13]:
pubs_pm = []
for pub in pubs:
    pubs_pm.append((pub[0], None, pub[2], None))

# Define our projective measurements acting on each qubit.
angles = np.array([[0.0, 0.0, 0.5 * np.pi, 0.0, 0.5 * np.pi, 0.5 * np.pi],[1.2,0.0, 0.1,3.14,0.7,0.3]])
# Set the distributions of the shots among the PMs.
bias = np.array([[0.34, 0.33, 0.33], [0.7,0.15,0.15]])

# Define the PM-simulable POVM.
pmsim_implementation = RandomizedPMs(n_qubit=n_qubit, bias=bias, angles=angles)
pmsim_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_job = sampler.run(pubs_pm, povm=pmsim_implementation, shots=pmsim_shots)

## Results

Retrieve the result of the sampling and use it to estiamte some expectation values via the post-processor

In [14]:
pmsim_result = pmsim_job.result()

for i_state in range(len(pmsim_result)):
    pmsim_postprocessor = POVMPostprocessor(pmsim_result[i_state])
    pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs) for obs in set_observables]
    pmsim_shots = sum(pmsim_postprocessor.counts.values())

    n = int(np.ceil(np.log10(pmsim_shots)))
    print('Exact        PM-sim')
    print(f'             ({pmsim_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}   {abs(pmsim_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (4096 shots)
 2.244e+00      2.448e+00       9.1%
-1.221e+00     -1.276e+00       4.5%
-8.662e-01     -1.270e+00      46.6%
 9.594e-01      6.108e-01      36.3%
-1.975e-01      2.873e-01     245.5%
-7.249e-01     -9.019e-01      24.4%
 3.303e-01     -2.424e-01     173.4%
-1.498e-01     -6.468e-02      56.8%
 1.441e+00      5.662e-01      60.7%
-5.073e-01     -8.444e-01      66.5%
Exact        PM-sim
             (940036 shots)
 1.582e-01        1.271e-01    19.7%
 6.594e-02        7.282e-02    10.4%
 3.723e-01        3.777e-01     1.5%
 2.555e-01        2.819e-01    10.3%
-4.867e-01       -5.028e-01     3.3%
-7.440e-01       -7.532e-01     1.2%
 2.420e+00        2.421e+00     0.0%
 3.360e-01        3.108e-01     7.5%
-6.905e-02       -6.582e-02     4.7%
 8.534e-01        8.384e-01     1.8%
Exact        PM-sim
             (4096 shots)
 2.427e+00      2.580e+00       6.3%
-9.346e-02     -1.317e-02      85.9%
 4.495e-01      2.499e-01      44.4%
-2.687e-0

## Multi_jobs

In [15]:
pmsim_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_jobs = sampler.run(pubs_pm, povm=pmsim_implementation, shots=pmsim_shots, multi_job=True)

In [16]:
for i_state, pmsim_job in enumerate(pmsim_jobs):
    pmsim_result = pmsim_job.result()[0]
    pmsim_postprocessor = POVMPostprocessor(pmsim_result)
    pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs) for obs in set_observables]
    pmsim_shots = sum(pmsim_postprocessor.counts.values())

    n = int(np.ceil(np.log10(pmsim_shots)))
    print('Exact        PM-sim')
    print(f'             ({pmsim_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}   {abs(pmsim_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (4096 shots)
 2.244e+00      2.542e+00      13.2%
-1.221e+00     -1.345e+00      10.2%
-8.662e-01     -8.733e-01       0.8%
 9.594e-01      8.398e-01      12.5%
-1.975e-01      4.123e-02     120.9%
-7.249e-01     -6.152e-01      15.1%
 3.303e-01      3.839e-01      16.2%
-1.498e-01     -5.566e-02      62.8%
 1.441e+00      1.397e+00       3.1%
-5.073e-01     -4.909e-01       3.2%
Exact        PM-sim
             (940036 shots)
 1.582e-01        1.712e-01     8.2%
 6.594e-02        5.346e-02    18.9%
 3.723e-01        3.658e-01     1.7%
 2.555e-01        2.520e-01     1.4%
-4.867e-01       -4.609e-01     5.3%
-7.440e-01       -7.401e-01     0.5%
 2.420e+00        2.411e+00     0.4%
 3.360e-01        2.986e-01    11.1%
-6.905e-02       -1.143e-01    65.5%
 8.534e-01        8.285e-01     2.9%
Exact        PM-sim
             (4096 shots)
 2.427e+00      2.533e+00       4.4%
-9.346e-02      3.735e-01     499.6%
 4.495e-01      6.743e-01      50.0%
-2.687e-0